In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pyperclip
from bs4 import BeautifulSoup

driver = webdriver.Chrome('driver/chromedriver')
driver.get("https://nid.naver.com/nidlogin.login")
time.sleep(1)


# id, pw 입력할 곳을 찾습니다.
tag_id = driver.find_element_by_id("id")
tag_pw = driver.find_element_by_id("pw")
tag_id.clear()
time.sleep(1)

# id 입력
tag_id.click()
pyperclip.copy('namsik')
tag_id.send_keys(Keys.CONTROL, 'v')
time.sleep(1)

# pw 입력
tag_pw.click()
pyperclip.copy('namju2620')
tag_pw.send_keys(Keys.CONTROL, 'v')
time.sleep(1)

# 로그인 버튼을 클릭합니다
login_btn = driver.find_element_by_id('log.login')
login_btn.click()
time.sleep(3)

In [2]:
test_list = []

for page_num in range(1, 100):
    base_url = "https://cafe.naver.com/ArticleList.nhn?search.clubid=28326943&search.boardtype=L&search.page={0}".format(page_num)
    time.sleep(0.5)
    driver.get(base_url)
    driver.switch_to.frame('cafe_main')
    
    html = driver.page_source
    bs = BeautifulSoup(html, 'lxml')
    
    # 작성 게시
    td_results = bs.find_all('td', class_ = 'td_article')
    test_list.append(td_results)    

In [3]:
result_ban = []
result_url = []

for page in test_list:
    for td in page:
        tmp = str(td)
        idx = tmp.find('학부모')
        
        if idx > 1:
            tmp_td = tmp[idx - 4:idx].strip()
            result_ban.append(tmp_td)
            
            idx2 = tmp.find('href="/ArticleRead.nhn?clubid=28326943')
            idx3 = tmp.find('onclick="clickcr(this, ')
            
            tmp_url = 'https://cafe.naver.com' + tmp[idx2 + 6 : idx3 - 2].replace("&amp;", "&").strip()
            
            result_url.append(tmp_url)

In [68]:
contents_list = []
for url in result_url:
    driver.get(url)
    time.sleep(1)
    driver.switch_to.frame('cafe_main')
    
    html = driver.page_source
    bs = BeautifulSoup(html, 'html.parser')
    text = ""
    for el in bs.find_all('div', class_ = 'se-module se-module-text'):
        text += el.get_text() + " "
        
    contents_list.append(text.replace("\n", ""))

KeyboardInterrupt: 

In [5]:
import pandas as pd
df = pd.DataFrame({"BAN":result_ban, "URL":result_url, "TEXT":contents_list})
df

,BAN,URL,TEXT
0,EU반,https://cafe.naver.com/ArticleRead.nhn?clubid=...,추석인데도 새로운 도전을 위해 멀리 있는 우리 예인이에게 무슨 기쁨을 줄 수 있을까...
1,TG반,https://cafe.naver.com/ArticleRead.nhn?clubid=...,울 아들 마음이 어찌 이렇게도 이쁜지ㆍ할머니 계속 3번이나 장난 전화줄알고 막...
2,EH반,https://cafe.naver.com/ArticleRead.nhn?clubid=...,밤새 안녕?ㅋㅋ추석인데 맛있는것두 못먹었지? 엄마도 선영이 없어서 입맛이 돌지 않아...
3,EU반,https://cafe.naver.com/ArticleRead.nhn?clubid=...,수진아어제 추석이었지송편은 주드나올해는 울 찐이도 없고 쪼매쓸쓸했어수진이도 그랬지 ...
4,EO반,https://cafe.naver.com/ArticleRead.nhn?clubid=...,추석은 경건하게 보내구있는 굉이들살쪘다 너무 많이 먹어서 그런가?실컷 먹었다 엄청 ...
...,...,...,...
1353,EH반,https://cafe.naver.com/ArticleRead.nhn?clubid=...,
1354,EH반,https://cafe.naver.com/ArticleRead.nhn?clubid=...,
1355,EH반,https://cafe.naver.com/ArticleRead.nhn?clubid=...,
1356,EH반,https://cafe.naver.com/ArticleRead.nhn?clubid=...,


In [36]:
data = df[df['TEXT'].str.len() > 0]

In [37]:
data.shape

(975, 3)

In [39]:
data['BAN'].value_counts()

EO반    213
EH반    181
TM반    162
ER반    104
TC반     79
TG반     77
EU반     74
TA반     49
TI반     36
Name: BAN, dtype: int64

In [56]:
word_list = []
for row in data['TEXT']:
    words = row.split(' ')
    for word in words:
        if len(word.strip()) > 0:
            word_list.append(word)

In [65]:
word_count = dict()
for i in word_list:
    word_count[i] = word_count.get(i, 0) + 1
    
word_count_sort = sorted(word_count.items(), key=lambda x:-x[1])[:10]    

In [67]:
word_count_sort

[('잘', 978),
 ('많이', 440),
 ('더', 395),
 ('오늘', 318),
 ('좀', 294),
 ('엄마가', 294),
 ('너무', 285),
 ('하고', 281),
 ('수', 277),
 ('다', 234)]